In [11]:
def probabilityEven():
    evenNumbers = [2, 4, 6]
    totalOutcomes = 6
    return len(evenNumbers) / totalOutcomes

def probabilityGreaterThan4():
    numbersGreaterThan4 = [5, 6]
    totalOutcomes = 6
    return len(numbersGreaterThan4) / totalOutcomes

def probabilityLessThan3():
    numbersLessThan3 = [1, 2]
    totalOutcomes = 6
    return len(numbersLessThan3) / totalOutcomes

def probabilitySumGreaterThanOrEqualTo7():
    favorableSums = [(1, 6), (2, 5), (3, 4), (4, 3), (5, 2), (6, 1),
                     (2, 6), (3, 5), (4, 4), (5, 3), (6, 2),
                     (3, 6), (4, 5), (5, 4), (6, 3),
                     (4, 6), (5, 5), (6, 4),
                     (5, 6), (6, 5), (6, 6)]
    totalOutcomes = 36
    return len(favorableSums) / totalOutcomes

def probabilitySumExactly8():
    favorableSums = [(2, 6), (3, 5), (4, 4), (5, 3), (6, 2)]
    totalOutcomes = 36
    return len(favorableSums) / totalOutcomes

def probabilitySecondDieOddGivenFirstDieGreaterThan4():
    favorableOutcomes = [1, 3, 5]
    totalOutcomes = 6
    return len(favorableOutcomes) / totalOutcomes

print(f"Probability of even number: {probabilityEven()}")
print(f"Probability of number greater than 4: {probabilityGreaterThan4()}")
print(f"Probability of number less than 3: {probabilityLessThan3()}")
print(f"Probability of sum of dice >= 7: {probabilitySumGreaterThanOrEqualTo7()}")
print(f"Probability of sum exactly 8: {probabilitySumExactly8()}")
print(f"Probability of second die odd given first die > 4: {probabilitySecondDieOddGivenFirstDieGreaterThan4()}")


Probability of even number: 0.5
Probability of number greater than 4: 0.3333333333333333
Probability of number less than 3: 0.3333333333333333
Probability of sum of dice >= 7: 0.5833333333333334
Probability of sum exactly 8: 0.1388888888888889
Probability of second die odd given first die > 4: 0.5


In [12]:
def probabilityRedCard():
    redCards = 26
    totalCards = 52
    return redCards / totalCards

def probabilityHeartGivenRedCard():
    hearts = 13
    redCards = 26
    return hearts / redCards

def probabilityDiamondGivenFaceCard():
    faceCards = 12
    diamonds = 3
    return diamonds / faceCards

def probabilitySpadeOrQueenGivenFaceCard():
    faceCards = 12
    spades = 3
    queens = 4
    return (spades + queens) / faceCards

print(f"Probability of drawing a red card: {probabilityRedCard()}")
print(f"Probability of heart given red card: {probabilityHeartGivenRedCard()}")
print(f"Probability of diamond given face card: {probabilityDiamondGivenFaceCard()}")
print(f"Probability of spade or queen given face card: {probabilitySpadeOrQueenGivenFaceCard()}")

Probability of drawing a red card: 0.5
Probability of heart given red card: 0.5
Probability of diamond given face card: 0.25
Probability of spade or queen given face card: 0.5833333333333334


In [17]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

def createBayesianNetwork():
    model = DiscreteBayesianNetwork([('Fever', 'Disease'), ('Cough', 'Disease'), ('Fatigue', 'Disease')])

    cpd_fever = TabularCPD(variable='Fever', variable_card=2, values=[[0.7], [0.3]])
    cpd_cough = TabularCPD(variable='Cough', variable_card=2, values=[[0.6], [0.4]])
    cpd_fatigue = TabularCPD(variable='Fatigue', variable_card=2, values=[[0.8], [0.2]])

    cpd_disease = TabularCPD(variable='Disease', variable_card=2,
                             values=[[0.9, 0.7, 0.6, 0.3, 0.8, 0.6, 0.5, 0.2],
                                     [0.1, 0.3, 0.4, 0.7, 0.2, 0.4, 0.5, 0.8]],
                             evidence=['Fever', 'Cough', 'Fatigue'],
                             evidence_card=[2, 2, 2])

    model.add_cpds(cpd_fever, cpd_cough, cpd_fatigue, cpd_disease)
    model.check_model()

    return model

def inferDiseaseGivenSymptoms(model, symptoms):
    inference = VariableElimination(model)
    query = inference.query(variables=['Disease'], evidence=symptoms)
    return query

model = createBayesianNetwork()
symptoms = {'Fever': 1, 'Cough': 1, 'Fatigue': 0}
query = inferDiseaseGivenSymptoms(model, symptoms)
print(f"Probability of Disease given symptoms: {query}")


Probability of Disease given symptoms: +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.5000 |
+------------+----------------+
| Disease(1) |         0.5000 |
+------------+----------------+


In [19]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

def createWeatherModel():
    model = BayesianNetwork([('Sunny', 'Weather'), ('Cloudy', 'Weather'), ('Rainy', 'Weather')])

    cpd_sunny = TabularCPD(variable='Sunny', variable_card=3, values=[[0.7], [0.2], [0.1]])
    cpd_cloudy = TabularCPD(variable='Cloudy', variable_card=3, values=[[0.3], [0.4], [0.3]])
    cpd_rainy = TabularCPD(variable='Rainy', variable_card=3, values=[[0.2], [0.3], [0.5]])

    model.add_cpds(cpd_sunny, cpd_cloudy, cpd_rainy)
    model.check_model()

    return model

def simulateWeatherFor10Days():
    weatherStates = ['Sunny', 'Cloudy', 'Rainy']
    initialState = 0  # Start with Sunny
    weatherHistory = [weatherStates[initialState]]

    transitionMatrix = np.array([[0.7, 0.2, 0.1],
                                 [0.3, 0.4, 0.3],
                                 [0.2, 0.3, 0.5]])

    for _ in range(9):
        nextState = np.random.choice([0, 1, 2], p=transitionMatrix[initialState])
        weatherHistory.append(weatherStates[nextState])
        initialState = nextState

    return weatherHistory

def probabilityAtLeast3RainyDays(weatherHistory):
    rainyDays = weatherHistory.count('Rainy')
    return rainyDays >= 3

simulatedWeather = simulateWeatherFor10Days()
print(f"Simulated Weather for 10 days: {simulatedWeather}")
print(f"Probability of at least 3 rainy days: {probabilityAtLeast3RainyDays(simulatedWeather)}")


Simulated Weather for 10 days: ['Sunny', 'Rainy', 'Rainy', 'Rainy', 'Rainy', 'Cloudy', 'Sunny', 'Rainy', 'Rainy', 'Sunny']
Probability of at least 3 rainy days: True
